In [1]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
from pprint import pprint

In [2]:
# Import API

In [3]:
## Get API Keys
load_dotenv("./environment/.env")

rapid_api_key = os.getenv("rapid_key")
rapid_api_host = os.getenv("rapid_host")

type(rapid_api_key)

cb07c5da44msh5da7167914d0ae7p1acdc5jsna1515709e811


In [4]:
## Zillow currunt ---------------------------------------------------------------------------------------------------

In [5]:
### Local & constants for zillow API
home_type = "Houses"
city = "philadelphia"
state = "pa"
search_str = city + "," + state

print( f"search string : {home_type} in {search_str}")

search string : Houses in philadelphia,pa


In [6]:
### Get Data from the API found on rapidAPI with API function ExtendedSearch (This is un undirect acces to Zillow)

url_extended_search = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring_extended_search = {"location":search_str ,
                               "home_type":home_type}

headers = {
    'x-rapidapi-host': rapid_api_host,
    'x-rapidapi-key': rapid_api_key
    }

response_extended_search = requests.request("GET", url_extended_search, headers=headers, params=querystring_extended_search)

In [8]:
### Store data in a pickle: Zillow_data
filename = "Zillow_data"
outfile = open(filename, "wb")

pk.dump(response_extended_search,outfile)
outfile.close()

In [15]:
### Open Pickle to make a list ZPID for the houses currently for sale
# Open Pickle
filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

#Transform data to Json
z_for_sale_resp =new_dict.json()

# Transform Json data to DataFrame
current_zillow_df = pd.json_normalize(data=z_for_sale_resp["props"])
print('num of rows:', len(current_zillow_df))
print('num of columns:', len(current_zillow_df.columns))

# make a list of the zpid as key to extract the data from the API similar_search, every ZPID is unique
key = current_zillow_df["zpid"].tolist()

num of rows: 40
num of columns: 21


In [11]:
## Zillow similar search : similar recently sold ----------------------------------------------------------------------------------------------------

In [12]:
### Get Data from the API found on rapidAPI with API function SimilarSales (This is un undirect acces to Zillow)

url_similar_house = "https://zillow-com1.p.rapidapi.com/similarSales"

#Initiate a new dictionnary to contain the data that is goind to be extracted
zpid_dict = {}

for zpid in key:
    # Create a query with one of the ZPID of the list key
    querystring_similar_house = {"zpid":zpid}
    
    # Put the query in the API and save the data in the variable response_similar_house
    response_similar_house = requests.request("GET", url_similar_house, headers=headers, params = querystring_similar_house)
    
    # Add the extracted data in a dictionnary that contains all the calls to the API
    zpid_dict[zpid] = response_similar_house.json()

In [13]:
### Store data in a pickle : Zillow_old_data
filename_old = "Zillow_sold_data"
outfile = open(filename_old, "wb")

pk.dump(zpid_dict,outfile)
outfile.close()